In [13]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

In [16]:
a = np.array([98.5, 1, 2, 1, 5, 7])
e = np.array([99, 1, 2, 1, 5, 7])
b = np.array([99, 2, 3, 2, 6, 8])
c = np.array([98.5, 23.6, 56.7])
d = np.array([43.2, 89.3, 18.4])

In [17]:
cosine = cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))
print(cosine)

[[0.99975724]]


In [18]:
cosine = cosine_similarity(a.reshape(1, -1), e.reshape(1, -1))
print(cosine)

[[0.9999999]]


In [5]:
cosine = cosine_similarity(c.reshape(1, -1), d.reshape(1, -1))
print(cosine)

[[0.63237215]]


In [30]:
a = set([98.5, 1, 2, 1, 5, 7])
e = set([99, 1, 2, 1, 5, 7])
b = set([99, 2, 3, 2, 6, 8])

In [31]:
shared = a.intersection(e)
shared

{1, 2, 5, 7}

In [32]:
total = a.union(e)
total

{1, 2, 5, 7, 98.5, 99}

In [34]:
len(shared) / len(total)

0.6666666666666666

In [150]:
a = "98.5 4 Ленина 5"
b = "99 4 Ленина 5"

In [151]:
import difflib

def similarity(s1, s2):
    normalized_1 = s1.lower()
    normalized_2 = s2.lower()
    matcher = difflib.SequenceMatcher(None, normalized_1, normalized_2)
    return matcher.ratio()

In [64]:
similarity(a, b)

0.8571428571428571

# Функция поиска дубликатов

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import difflib

def similarity(s1, s2):
    normalized_1 = s1.lower()
    normalized_2 = s2.lower()
    matcher = difflib.SequenceMatcher(None, normalized_1, normalized_2)
    return matcher.ratio()

def search_duplicates_1(sim, sq, treshold, iid):                                                         #sim = df['Сцепка_2'], sq = df['ОПлощадь']
    '''
    Перед использованием функции необходимо убрать все мусорные объекты и
    избавиться от прямых дублей по минимальному набору:
    Цена, площадь, этаж, сцепка по координатам.
    sim - Сцепка_3 (этаж координаты)
    sq - параметр площади
    pr - параметр цены/удельной цены
    treshold_s - порог по площади (абсолютный, кв.м)
    treshold_pr - порог по цене (доля)
    iid - ID объекта
    keep - метод для выделения дубликатов
    '''
    treshold = treshold                                                                                  #порог по площади (<=)
    matrix = np.zeros((len(sq), len(sq)))                                                                #создаем нулевую матрицу
    
    df_sq_sim = pd.DataFrame({'ID': iid, 'sq': sq, 'sim': sim})                                          #создаем DF для площади и сцепки
    df_sq_sim['dup'] = df_sq_sim['sim'].duplicated(keep = False)                                         #создаем столбик с дубликатами по сцепке
    duplicates = df_sq_sim[df_sq_sim['dup'] == True]                                                     #сортируем все дубликаты по сцепке

    df_sq = pd.concat([pd.DataFrame(duplicates['sq']).T] * len(duplicates), ignore_index = True)         #матрица для площадей
    df_sq.columns = range(0, len(duplicates))
    
    df_sim = pd.concat([pd.DataFrame(duplicates['sim']).T] * len(duplicates), ignore_index = True)       #матрица для сцепок
    df_sim.columns = range(0, len(duplicates))
    
    sim = np.array(duplicates['sim'])
    sq = np.array(duplicates['sq'])

    n = 0
    for i in tqdm(range(0, len(sq), 1), position = 0, leave = True):                                     #перебираем столбцы патриц df_sim, df_sq
        n = i + 1
        for j in range(0, n, 1):
            #similar = similarity(sim[j], df_sim[i][j])                                                  #сходство по сцепке через функцию
            diff = abs(sq[j] - df_sq[i][j])                                                              #разница по площади
            #print(diff)
            #if similar == 1 and diff <= treshold:
            if (sim[j] == df_sim[i][j]) and (diff <= treshold):                                            #прямое сходство
                matrix[j][i] = 1                                                                         #строка, столбец
                matrix[i][j] = 1                                                                         #строка, столбец - зеркально
            else:
                matrix[j][i] = 0                                                                         #строка, столбец
                matrix[i][j] = 0                                                                         #строка, столбец - зеркально
    
    summ = []                                                                                            #считаем суммы по строкам матрицы
    for i in range(0, len(sq), 1):
        summ.append(matrix[i].sum() - 1)                                                                 #вычитаем 1 из диагонали матрицы
    
    duplicates['label'] = summ                                                                           #добавляем метку в DF, label показывает количество пересечений объекта с другими                                    
    
    return duplicates

In [3]:
df = pd.read_excel('D:\Стас\Ученье - Свет\Оценка\Аспирантура\Видео по NN, Data mining, ABC и т.д\Распределение Гаусса, регрессия\Python\!Наработки\Квартиры_Владивосток/Выборка.xlsx', sheet_name = 0)
df

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Цена(кв.м),dup_all,dup_1,dup_2,...,Расположение на этаже,Срок сдачи,Выдача ключей,Отделка,Название новостройки,Тип участия,Название ЖК,Подъезд,Способ продажи,Вид сделки
0,1_этажи,0,этажи,15.955577,8500000,9500000.0,228494.623656,1,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2_этажи,1,этажи,15.990262,8800000,NaN,169230.769231,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3_этажи,0,этажи,15.747032,6900000,7100000.0,128731.343284,1,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4_этажи,0,этажи,16.627921,16650000,16800000.0,185825.892857,1,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_этажи,0,этажи,15.894952,8000000,NaN,96735.187424,1,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8957,3802_фарпост,0,фарпост,15.876024,7850000,NaN,150961.538462,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8958,3803_фарпост,0,фарпост,16.308716,12100000,NaN,181681.681682,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8959,3804_фарпост,0,фарпост,15.729488,6780000,NaN,233793.103448,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8960,3805_фарпост,0,фарпост,16.606676,16300000,NaN,207643.312102,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df_1 = df[df['Мусор'] == 0]
df_1 = df_1[df_1['dup_1'] == 0]
df_1

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Цена(кв.м),dup_all,dup_1,dup_2,...,Расположение на этаже,Срок сдачи,Выдача ключей,Отделка,Название новостройки,Тип участия,Название ЖК,Подъезд,Способ продажи,Вид сделки
0,1_этажи,0,этажи,15.955577,8500000,9500000.0,228494.623656,1,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3_этажи,0,этажи,15.747032,6900000,7100000.0,128731.343284,1,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4_этажи,0,этажи,16.627921,16650000,16800000.0,185825.892857,1,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_этажи,0,этажи,15.894952,8000000,NaN,96735.187424,1,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6_этажи,0,этажи,15.607270,6000000,NaN,137299.771167,1,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8953,3798_фарпост,0,фарпост,16.001562,8900000,NaN,211904.761905,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8958,3803_фарпост,0,фарпост,16.308716,12100000,NaN,181681.681682,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8959,3804_фарпост,0,фарпост,15.729488,6780000,NaN,233793.103448,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8960,3805_фарпост,0,фарпост,16.606676,16300000,NaN,207643.312102,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import difflib

def similarity(s1, s2):
    normalized_1 = s1.lower()
    normalized_2 = s2.lower()
    matcher = difflib.SequenceMatcher(None, normalized_1, normalized_2)
    return matcher.ratio()

def search_duplicates_2(sim, sq, pr, treshold_s, treshold_pr, iid, keep):                                #sim = df['Сцепка_3'], sq = df['ОПлощадь'], keep = first or last
    '''
    Перед использованием функции необходимо убрать все мусорные объекты и
    избавиться от прямых дублей по минимальному набору:
    Цена, площадь, этаж, сцепка по координатам.
    sim - Сцепка_3 (этаж координаты)
    sq - параметр площади
    pr - параметр цены/удельной цены
    treshold_s - порог по площади (абсолютный, кв.м)
    treshold_pr - порог по цене (доля)
    iid - ID объекта
    keep - метод для выделения дубликатов
    '''
    treshold_s = treshold_s                                                                              #порог по площади (<=)
    treshold_pr = treshold_pr                                                                            #порог по цене (<=)
    matrix = np.zeros((len(sq), len(sq)))                                                                #создаем нулевую матрицу
    
    df_sq_sim = pd.DataFrame({'ID': iid, 'price': pr, 'sq': sq, 'sim': sim})                             #создаем DF для площади и сцепки
    df_sq_sim['dup'] = df_sq_sim['sim'].duplicated(keep = False)                                         #создаем столбик с дубликатами по сцепке
    duplicates = df_sq_sim[df_sq_sim['dup'] == True]                                                     #сортируем все дубликаты по сцепке

    df_sq = pd.concat([pd.DataFrame(duplicates['sq']).T] * len(duplicates), ignore_index = True)         #матрица для площадей
    df_sq.columns = range(0, len(duplicates))
    
    df_sim = pd.concat([pd.DataFrame(duplicates['sim']).T] * len(duplicates), ignore_index = True)       #матрица для сцепок
    df_sim.columns = range(0, len(duplicates))
    
    df_pr = pd.concat([pd.DataFrame(duplicates['price']).T] * len(duplicates), ignore_index = True)      #матрица для цены
    df_pr.columns = range(0, len(duplicates))
    
    sim = np.array(duplicates['sim'])
    sq = np.array(duplicates['sq'])
    pr = np.array(duplicates['price'])

    n = 0
    for i in tqdm(range(0, len(sq), 1), position = 0, leave = True):                                     #перебираем столбцы патриц df_sim, df_sq
        n = i + 1
        for j in range(0, n, 1):
            #similar = similarity(sim[j], df_sim[i][j])                                                  #сходство по сцепке через функцию
            diff_pr = abs(min(pr[j], df_pr[i][j]) / max(pr[j], df_pr[i][j]) - 1)                         #разница по цене
            diff_s = abs(sq[j] - df_sq[i][j])                                                            #разница по площади
            #print(diff_pr, diff_s)
            #print(diff)
            #if similar == 1 and diff <= treshold:
            if (sim[j] == df_sim[i][j]) and (diff_s <= treshold_s) and (diff_pr <= treshold_pr):         #прямое сходство
                matrix[j][i] = 1                                                                         #строка, столбец
                matrix[i][j] = 1                                                                         #строка, столбец - зеркально
            else:
                matrix[j][i] = 0                                                                         #строка, столбец
                matrix[i][j] = 0                                                                         #строка, столбец - зеркально
    
    summ = []                                                                                            #считаем суммы по строкам матрицы
    for i in range(0, len(sq), 1):
        summ.append(matrix[i].sum() - 1)                                                                 #вычитаем 1 из диагонали матрицы
    
    duplicates['label'] = summ                                                                           #добавляем метку в DF, label показывает количество пересечений объекта с другими  
    duplicates = duplicates[duplicates['label'] != 0]
    dup_i = duplicates['sim'].duplicated(keep = keep)
    duplicates['dup_i'] = dup_i
    
    return duplicates

In [100]:
df_dup = search_duplicates_2(df_1['Сцепка_3'], df_1['ОПлощадь'], df_1['Цена(кв.м)'], 2, 0.03, df_1['ID'], keep = 'last')
df_dup

100%|██████████| 3078/3078 [02:58<00:00, 17.20it/s]
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,price,sq,sim,dup,label,dup_i
7,8_этажи,125961.538462,52.0,4 131.958015 43.105124,True,1.0,True
9,10_этажи,211764.705882,17.0,8 131.945124 43.131315,True,1.0,True
10,11_этажи,204322.200393,50.9,7 131.855804 43.08729,True,3.0,True
17,18_этажи,201067.615658,28.1,20 131.920627 43.193734,True,1.0,True
19,20_этажи,189405.286344,45.4,4 131.934506 43.114588,True,1.0,True
...,...,...,...,...,...,...,...
8898,3743_фарпост,125000.000000,64.0,9 131.918228 43.091774,True,1.0,False
8918,3763_фарпост,133843.212237,52.3,1 131.945762 43.099252,True,1.0,False
8920,3765_фарпост,209024.390244,41.0,1 131.890111 43.123452,True,1.0,False
8952,3797_фарпост,138974.025974,77.0,1 131.861275 43.098383,True,1.0,False


In [101]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Дубликейт.xlsx')
df_dup.to_excel(writer, 'data')
writer.save()

# Поиск неточностей

In [49]:
df = pd.read_excel('Выборка.xlsx', sheet_name = 0)
df

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Цена(кв.м),Выброс_2,Выброс_1,Комм,...,Расположение на этаже,Срок сдачи,Выдача ключей,Отделка,Название новостройки,Тип участия,Название ЖК,Подъезд,Способ продажи,Вид сделки
0,1_этажи,0,этажи,15.955577,8500000,9500000.0,228494.623656,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2_этажи,1,этажи,15.990262,8800000,NaN,169230.769231,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3_этажи,0,этажи,15.747032,6900000,7100000.0,128731.343284,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4_этажи,0,этажи,16.627921,16650000,16800000.0,185825.892857,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_этажи,0,этажи,15.894952,8000000,NaN,96735.187424,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8957,3802_фарпост,0,фарпост,15.876024,7850000,NaN,150961.538462,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8958,3803_фарпост,0,фарпост,16.308716,12100000,NaN,181681.681682,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8959,3804_фарпост,0,фарпост,15.729488,6780000,NaN,233793.103448,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8960,3805_фарпост,0,фарпост,16.606676,16300000,NaN,207643.312102,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_1 = df[df['Мусор'] == 0]
df_1 = df_1[df_1['dup_res'] == 0]
df_1

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Цена(кв.м),Выброс_2,Выброс_1,Комм,...,Расположение на этаже,Срок сдачи,Выдача ключей,Отделка,Название новостройки,Тип участия,Название ЖК,Подъезд,Способ продажи,Вид сделки
32,33_этажи,0,этажи,15.039286,3400000,3600000.0,203592.814371,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,58_этажи,0,этажи,15.869634,7800000,NaN,162500.000000,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,71_этажи,0,этажи,15.830414,7500000,NaN,208913.649025,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,85_этажи,0,этажи,15.640060,6200000,NaN,177142.857143,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,105_этажи,0,этажи,15.444751,5100000,5150000.0,146131.805158,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8957,3802_фарпост,0,фарпост,15.876024,7850000,NaN,150961.538462,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8958,3803_фарпост,0,фарпост,16.308716,12100000,NaN,181681.681682,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8959,3804_фарпост,0,фарпост,15.729488,6780000,NaN,233793.103448,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8960,3805_фарпост,0,фарпост,16.606676,16300000,NaN,207643.312102,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
def search_inaccuracy(hitch, val, iid):
    '''
    hitch - Сцепка_к (координаты), если параметр относится к дому и Сцепка_3, если параметр относится к квартире
    val - параметр проверки (этажность, год, материал стен или отделка)
    iid - ID объекта
    '''
    df_hitch_val = pd.DataFrame({'ID': iid, 'hitch': hitch, 'val': val})                                                    #создаем DF для значения (например, этажность или год) и сцепки
    df_hitch_val['hitch_count'] = df_hitch_val['hitch'].map(dict(df_hitch_val['hitch'].value_counts(ascending = False)))    #считаем количество квартир по каждой сцепке и создаем словарь
    inaccuracy = df_hitch_val[df_hitch_val['hitch_count'] != 1]                                                             #исключаем уникальные сцепки
    inaccuracy['hitch_summ'] = inaccuracy['hitch'].map(dict(inaccuracy.groupby('hitch')['val'].sum()))                      #находим суммы по каждой сцепке
    inaccuracy['hitch_mult'] = inaccuracy['val'] * inaccuracy['hitch_count']                                                #находим произведение количества и балла
    inaccuracy['Метка'] = [True if i == j else False for i, j in zip(inaccuracy['hitch_summ'], inaccuracy['hitch_mult'])]   #сравниваем сумму и произведение, если не равны, то False
    return inaccuracy

In [39]:
df_inacc = search_inaccuracy(df_1['Сцепка_к'], df_1['Этажность_1'], df_1['ID'])
df_inacc

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,ID,hitch,val,hitch_count,hitch_summ,hitch_mult,Метка
32,33_этажи,131.965561 43.093268,9.0,6,54.0,54.0,True
57,58_этажи,132.064474 43.298764,9.0,4,36.0,36.0,True
70,71_этажи,131.929385 43.177977,10.0,6,60.0,60.0,True
84,85_этажи,131.920627 43.193734,26.0,72,1884.0,1872.0,False
134,135_этажи,131.940093 43.175795,22.0,27,605.0,594.0,False
...,...,...,...,...,...,...,...
8957,3802_фарпост,131.936752 43.173388,9.0,2,18.0,18.0,True
8958,3803_фарпост,131.913216 43.129407,9.0,5,45.0,45.0,True
8959,3804_фарпост,131.961698 43.17016,17.0,8,140.0,136.0,False
8960,3805_фарпост,131.906164 43.128611,16.0,5,80.0,80.0,True


In [40]:
df_inacc[df_inacc['Метка'] == False]

,ID,hitch,val,hitch_count,hitch_summ,hitch_mult,Метка
84,85_этажи,131.920627 43.193734,26.0,72,1884.0,1872.0,False
134,135_этажи,131.940093 43.175795,22.0,27,605.0,594.0,False
221,222_этажи,131.970295 43.117865,25.0,11,277.0,275.0,False
262,263_этажи,131.929906 43.176866,10.0,8,77.0,80.0,False
268,269_этажи,131.961949 43.102076,25.0,14,349.0,350.0,False
...,...,...,...,...,...,...,...
8921,3766_фарпост,132.000631 43.231114,25.0,18,451.0,450.0,False
8928,3773_фарпост,131.920627 43.193734,26.0,72,1884.0,1872.0,False
8935,3780_фарпост,131.920627 43.193734,26.0,72,1884.0,1872.0,False
8942,3787_фарпост,131.961338 43.086783,24.0,8,195.0,192.0,False


In [42]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Неточности_этаж.xlsx')
df_inacc.to_excel(writer, 'data')
writer.save()

In [43]:
df_inacc = search_inaccuracy(df_1['Сцепка_к'], df_1['ГП_2'], df_1['ID'])
df_inacc

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,ID,hitch,val,hitch_count,hitch_summ,hitch_mult,Метка
32,33_этажи,131.965561 43.093268,1975,6,11850,11850,True
57,58_этажи,132.064474 43.298764,1981,4,7924,7924,True
70,71_этажи,131.929385 43.177977,2008,6,12048,12048,True
84,85_этажи,131.920627 43.193734,2023,72,145656,145656,True
134,135_этажи,131.940093 43.175795,2022,27,54594,54594,True
...,...,...,...,...,...,...,...
8957,3802_фарпост,131.936752 43.173388,1989,2,3978,3978,True
8958,3803_фарпост,131.913216 43.129407,1984,5,9920,9920,True
8959,3804_фарпост,131.961698 43.17016,2020,8,16160,16160,True
8960,3805_фарпост,131.906164 43.128611,2007,5,10035,10035,True


In [44]:
df_inacc[df_inacc['Метка'] == False]

,ID,hitch,val,hitch_count,hitch_summ,hitch_mult,Метка
330,331_этажи,132.025658 43.263639,2021,5,10104,10105,False
366,367_этажи,131.925505 43.175992,2001,12,24015,24012,False
1279,1280_этажи,132.025658 43.263639,2021,5,10104,10105,False
1687,1688_этажи,132.025658 43.263639,2021,5,10104,10105,False
1989,1990_этажи,131.999607 43.230615,2023,64,129469,129472,False
...,...,...,...,...,...,...,...
8508,3353_фарпост,131.913207 43.177057,2016,8,16127,16128,False
8593,3438_фарпост,131.925505 43.175992,2001,12,24015,24012,False
8813,3658_фарпост,131.996031 43.227573,2004,5,9969,10020,False
8814,3659_фарпост,131.894845 43.128466,1979,4,7924,7916,False


In [45]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Неточности_год.xlsx')
df_inacc.to_excel(writer, 'data')
writer.save()

In [67]:
df_dup = search_duplicates(df['Сцепка_3'], df['ОПлощадь'], 2, df['ID'])
df_dup

100%|██████████| 2860/2860 [01:16<00:00, 37.36it/s]
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,sq,sim,dup,label
5,8_этажи,52.0,4 131.958015 43.105124,True,1.0
7,10_этажи,17.0,8 131.945124 43.131315,True,1.0
8,11_этажи,50.9,7 131.855804 43.08729,True,3.0
10,14_этажи,28.7,4 132.028703 43.230562,True,1.0
13,18_этажи,28.1,20 131.920627 43.193734,True,1.0
...,...,...,...,...,...
5656,3789_фарпост,70.3,19 131.911177 43.119859,True,0.0
5660,3794_фарпост,42.8,18 131.957934 43.170232,True,0.0
5661,3796_фарпост,87.2,7 131.925199 43.121037,True,0.0
5662,3797_фарпост,77.0,1 131.861275 43.098383,True,1.0


In [69]:
df_dup_n = df_dup[df_dup['label'] != 0]
df_dup_n

,ID,sq,sim,dup,label
5,8_этажи,52.0,4 131.958015 43.105124,True,1.0
7,10_этажи,17.0,8 131.945124 43.131315,True,1.0
8,11_этажи,50.9,7 131.855804 43.08729,True,3.0
10,14_этажи,28.7,4 132.028703 43.230562,True,1.0
13,18_этажи,28.1,20 131.920627 43.193734,True,1.0
...,...,...,...,...,...
5643,3770_фарпост,71.3,9 131.998187 43.229334,True,2.0
5644,3771_фарпост,31.1,3 131.907754 43.151859,True,1.0
5647,3778_фарпост,34.0,2 131.86989 43.103899,True,1.0
5649,3780_фарпост,27.9,18 131.920627 43.193734,True,1.0


In [71]:
df_dup_n[df_dup_n['sim'] == "4 132.028703 43.230562"]

,ID,sq,sim,dup,label
10,14_этажи,28.7,4 132.028703 43.230562,True,1.0
2192,326_циан,28.7,4 132.028703 43.230562,True,1.0


In [72]:
df_dup_n['label'].max()

7.0

In [46]:
df_inacc = search_inaccuracy(df_1['Сцепка_к'], df_1['Число_жк'], df_1['ID'])
df_inacc

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,ID,hitch,val,hitch_count,hitch_summ,hitch_mult,Метка
32,33_этажи,131.965561 43.093268,3,6,18,18,True
57,58_этажи,132.064474 43.298764,3,4,12,12,True
70,71_этажи,131.929385 43.177977,3,6,18,18,True
84,85_этажи,131.920627 43.193734,3,72,216,216,True
134,135_этажи,131.940093 43.175795,3,27,81,81,True
...,...,...,...,...,...,...,...
8957,3802_фарпост,131.936752 43.173388,3,2,6,6,True
8958,3803_фарпост,131.913216 43.129407,3,5,15,15,True
8959,3804_фарпост,131.961698 43.17016,3,8,24,24,True
8960,3805_фарпост,131.906164 43.128611,3,5,15,15,True


In [47]:
df_inacc[df_inacc['Метка'] == False]

,ID,hitch,val,hitch_count,hitch_summ,hitch_mult,Метка
1715,1716_этажи,131.895213 43.120452,3,7,20,21,False
4453,1445_циан,131.895213 43.120452,3,7,20,21,False
5026,2018_циан,131.87572 43.107631,3,3,8,9,False
5028,2020_циан,131.881361 43.114641,3,4,10,12,False
5030,2022_циан,131.912811 43.115957,3,7,20,21,False
5139,2131_циан,131.998322 43.229938,2,3,7,6,False
5152,2144_циан,131.912811 43.115957,2,7,20,14,False
5326,171_фарпост,131.912811 43.115957,3,7,20,21,False
5644,489_фарпост,131.900262 43.101543,3,3,8,9,False
5745,590_фарпост,131.87572 43.107631,3,3,8,9,False


In [48]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Неточности_ЖК.xlsx')
df_inacc.to_excel(writer, 'data')
writer.save()

In [54]:
df_inacc = search_inaccuracy(df_1['Сцепка_к'], df_1['Число_c'], df_1['ID'])
df_inacc

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,ID,hitch,val,hitch_count,hitch_summ,hitch_mult,Метка
32,33_этажи,131.965561 43.093268,4,6,24,24,True
57,58_этажи,132.064474 43.298764,4,4,16,16,True
70,71_этажи,131.929385 43.177977,4,6,24,24,True
84,85_этажи,131.920627 43.193734,3,72,214,216,False
134,135_этажи,131.940093 43.175795,3,27,81,81,True
...,...,...,...,...,...,...,...
8957,3802_фарпост,131.936752 43.173388,2,2,4,4,True
8958,3803_фарпост,131.913216 43.129407,4,5,20,20,True
8959,3804_фарпост,131.961698 43.17016,3,8,23,24,False
8960,3805_фарпост,131.906164 43.128611,2,5,10,10,True


In [55]:
df_inacc[df_inacc['Метка'] == False]

,ID,hitch,val,hitch_count,hitch_summ,hitch_mult,Метка
84,85_этажи,131.920627 43.193734,3,72,214,216,False
202,203_этажи,131.949418 43.214092,2,9,24,18,False
268,269_этажи,131.961949 43.102076,3,14,41,42,False
330,331_этажи,132.025658 43.263639,2,5,11,10,False
421,422_этажи,131.928505 43.111199,2,3,7,6,False
...,...,...,...,...,...,...,...
8928,3773_фарпост,131.920627 43.193734,3,72,214,216,False
8935,3780_фарпост,131.920627 43.193734,3,72,214,216,False
8942,3787_фарпост,131.961338 43.086783,2,8,22,16,False
8956,3801_фарпост,131.930445 43.115687,4,9,30,36,False


In [56]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Неточности_стены.xlsx')
df_inacc.to_excel(writer, 'data')
writer.save()

In [53]:
list(df_1.columns)

['ID',
 'Мусор',
 'Агрегатор',
 'LN(Цена)',
 'Цена',
 'Цена_перв',
 'Цена(кв.м)',
 'Выброс_2',
 'Выброс_1',
 'Комм',
 'IsolationForest',
 'OneClassSVM',
 'Выброс_3',
 'Комм.1',
 'Исп',
 'dup_all',
 'dup_1',
 'dup_2',
 'dup(<=0.03)',
 'Пересечения(<=0.03)',
 'Пересечения(>=0.03)',
 'dup_res',
 'КН_гис',
 'ОПлощадь',
 'Оплощадь_перв',
 'Комнатность_1',
 'Комнатность_перв',
 'Этаж_1',
 'Этаж_1_в',
 'Этаж_2',
 'Этаж_3',
 'Этаж_4',
 'Этажность_1',
 'Этажность_2',
 'Неточность',
 'Этажность_гис',
 'Этажность_мин',
 'Этажность_прдом',
 'разница',
 'разница.1',
 'Этаж/Этажность',
 'ГП_2',
 'Неточность.1',
 'ГП_1',
 'ГП_гис',
 'ГП_мин',
 'Проверка',
 'ГП_прдом',
 'разница.2',
 'ГП_перв',
 'Ремонт_1',
 'Ремонт_2',
 'Ремонт_1_в',
 'Неточность.2',
 'Число_р',
 'Ремонт_перв',
 'Стены_2',
 'Стены_1',
 'Стены_1_в',
 'Неточность.3',
 'Число_c',
 'Стены_dom',
 'Проверка.1',
 'Стены_перв',
 'Санузел_1',
 'Санузел_перв',
 'Просмотры_1',
 'Просмотры_перв',
 'Дата',
 'Терраса',
 'Многоуровневая',
 'Витрины